In [1]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [31]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.utils import *
from fgc_support_retri import config
from evaluation.eval import eval_sp_fgc

In [3]:
def print_sent_ie(sp_i, d):
    print("sentence{}".format(sp_i))
    print(d['SENTS'][sp_i]['text'])
    for e in d['SENTS'][sp_i]['IE']['NER']:
        print(e)

In [72]:
data = json_load("prediction/sgroup_sfscore_4.json")

In [73]:
len(data)

193

In [74]:
# single_num = 0
# other_q = 0

# answer_in = 0
# answer_out = 0 
# for d in data:
#     for q in d['QUESTIONS']:
#         if q['QTYPE'] == '申論':
#             other_q += 1
        
#         elif 'Single-Span-Extraction' in q['AMODE']:
#             single_num += 1
#             sp_text = ""
#             for s_i, s in enumerate(d['SENTS']):
#                 if s_i in q['sp']:
#                     sp_text += s['text']
#             shint_text = ""
#             for s_i , s in enumerate(d['SENTS']):
#                 if s_i in q['SHINT']:
#                     shint_text += s['text']
#             for answer in q['ANSWER']:
#                 if answer['ATEXT_CN'] in shint_text:
#                     if answer['ATEXT_CN'] not in sp_text:
#                         answer_out += 1
#                         print(q['QID'])
#                         break
#         else:
#             other_q += 1

In [75]:
def find_q(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q)

In [76]:
data[8]['SENTS'][14]

{'text': '类似的集会运动出现在了华盛顿特区、旧金山、洛杉矶、芝加哥、波士顿、波特兰、阿布奎基、坦帕、夏洛特、西雅图、丹佛和缅因州波特兰。',
 'start': 329,
 'end': 392,
 'IE': {'NER': [{'id': 'D0-S0-M0',
    'string': '华盛顿',
    'type': 'STATE_OR_PROVINCE',
    'char_b': 11,
    'char_e': 14},
   {'id': 'D0-S0-M1',
    'string': '特区',
    'type': 'GPE',
    'char_b': 14,
    'char_e': 16},
   {'id': 'D0-S0-M2',
    'string': '旧金山',
    'type': 'CITY',
    'char_b': 17,
    'char_e': 20},
   {'id': 'D0-S0-M3',
    'string': '洛杉矶',
    'type': 'CITY',
    'char_b': 21,
    'char_e': 24},
   {'id': 'D0-S0-M4',
    'string': '芝加哥',
    'type': 'CITY',
    'char_b': 25,
    'char_e': 28},
   {'id': 'D0-S0-M5',
    'string': '波士顿',
    'type': 'CITY',
    'char_b': 29,
    'char_e': 32},
   {'id': 'D0-S0-M6',
    'string': '波特兰',
    'type': 'CITY',
    'char_b': 33,
    'char_e': 36},
   {'id': 'D0-S0-M7',
    'string': '阿布奎基',
    'type': 'PER',
    'char_b': 37,
    'char_e': 41},
   {'id': 'D0-S0-M8',
    'string': '夏洛特',
    'type': 'CITY',
    'char

In [77]:
all_sp_predictions = []
all_items = []
for d in data:
    for q in d['QUESTIONS']:
        all_items.append(q['SHINT'])
        all_sp_predictions.append(q['sp'])
        print(q['sp'])
        print([q['sp_scores'][sp_id] for sp_id in q['sp']])
#         print([q['sp_scores'][sp_id] for sp_id in [0, 1, 3, 14, 15]])

[2, 3, 4]
[0.5393716096878052, 0.5072380304336548, 0.6853436231613159]
[17, 18]
[0.5679527521133423, 0.7861303091049194]
[20]
[0.5789908766746521]
[18, 22, 25, 26]
[0.5302683711051941, 0.5583633780479431, 0.6507050395011902, 0.6839635372161865]
[12]
[0.5134865045547485]
[22, 26, 32]
[0.5245939493179321, 0.771422803401947, 0.5138259530067444]
[11, 13, 16]
[0.5175391435623169, 0.5136318206787109, 0.5832600593566895]
[0]
[0.7912471294403076]
[0, 1, 3, 14, 15]
[0.811569333076477, 0.7738611698150635, 0.6014977693557739, 0.5378933548927307, 0.6542473435401917]
[0]
[0.7814855575561523]
[0, 15, 16, 19]
[0.7884235978126526, 0.6948872804641724, 0.7147294282913208, 0.5647315382957458]
[0]
[0.7645222544670105]
[0, 9, 10, 11]
[0.712824285030365, 0.5969290137290955, 0.7399686574935913, 0.6690007448196411]
[0]
[0.6401932239532471]
[0, 6, 7, 8, 9]
[0.7647815346717834, 0.7343363761901855, 0.7183746099472046, 0.6480568051338196, 0.7134458422660828]
[0, 6, 7, 8, 9]
[0.7604073882102966, 0.7478870749473572

In [43]:
metrics = eval_sp_fgc(all_items, all_sp_predictions)

{'sp_em': 0.13471502590673576, 'sp_prec': 0.5818961263261784, 'sp_recall': 0.6720577350111029, 'sp_f1': 0.5679713205619944}


In [21]:
def print_analysis_from_qid(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q['QID'])
                print(q['QTEXT_CN'])
                print("atype:{}".format(q['ATYPE']))
                print("SHINT:{}".format(q['SHINT']))
                print("sp:{}".format(q['sp']))
                print("answer:{}".format(q['ANSWER']))
                all_set = set(q['SHINT']) | set(q['sp'])
                for sp_i in range(min(all_set), max(all_set) + 1):
                    print_sent_ie(sp_i, d)
                    print()

In [15]:
print_analysis_from_qid('D105Q01', data)

D105Q01
全民健保于哪一年实施?
atype:Date-Duration
SHINT:[0, 1, 35, 36]
sp:[1, 3, 4, 8, 9, 15, 36]
answer:[{'ATEXT': '1995年', 'ATEXT_CN': '1995年', 'ATOKEN': [{'text': '1995年', 'text_cn': '1995年', 'start': 510, 'end': 515}]}]
sentence0
全民健康保险，

sentence1
一般简称为「全民健保」或「健保」，

sentence2
是一种强制性保险的福利政策，
{'id': 'D0-S0-M0', 'string': '一', 'type': 'NUMBER', 'char_b': 1, 'char_e': 2}

sentence3
法源是依据《中华民国宪法增修条文》所实施的全民医疗保险制度。
{'id': 'D0-S0-M0', 'string': '中华民国宪法增修条文', 'type': 'MISC', 'char_b': 6, 'char_e': 16}

sentence4

在全民健康保险实施以前，

sentence5
台湾按照职业别分别实施军人保险（军保）、公教人员保险（公保）、劳工保险（劳保）、农民健康保险（农保），
{'id': 'D0-S0-M0', 'string': '台湾', 'type': 'STATE_OR_PROVINCE', 'char_b': 0, 'char_e': 2}
{'id': 'D0-S0-M1', 'string': '公保', 'type': 'PER', 'char_b': 27, 'char_e': 29}
{'id': 'D0-S0-M2', 'string': '农民', 'type': 'TITLE', 'char_b': 40, 'char_e': 42}

sentence6
并有保险业各自承保的健康保险。

sentence7
 1986年5月，
{'id': 'D0-S0-M0', 'string': '1986年5月', 'type': 'DATE', 'char_b': 1, 'char_e': 8}

sentence8
行政院核定的「中华民国台湾经济长期展望」中提到以2000年为

In [33]:
d = data[2]
for q in d['QUESTIONS']:
    print(q['QID'])
    print(q['QTEXT_CN'])
    print("atype:{}".format(q['ATYPE']))
    print("SHINT:{}".format(q['SHINT']))
    print("sp:{}".format(q['sp']))
    all_set = set(q['SHINT']) | set(q['sp'])
    for sp_i in range(min(all_set), max(all_set) + 1):
        print_sent_ie(sp_i, d)
        print()

D003Q03
在「乌台诗案」中，有哪些人出面救了苏东坡？
atype:Person
SHINT:[19, 20, 21]
sp:[15, 20]
sentence15
苏轼参加救灾。

sentence16

元丰二年（1079年），
{'id': 'D0-S0-M0', 'string': '元丰', 'type': 'MISC', 'char_b': 1, 'char_e': 3}
{'id': 'D0-S0-M1', 'string': '二年', 'type': 'DATE', 'char_b': 3, 'char_e': 5}
{'id': 'D0-S0-M2', 'string': '1079年', 'type': 'DATE', 'char_b': 6, 'char_e': 11}

sentence17
四十三岁时，
{'id': 'D0-S0-M0', 'string': '四十三', 'type': 'NUMBER', 'char_b': 0, 'char_e': 3}
{'id': 'D0-S0-M1', 'string': '岁', 'type': 'MISC', 'char_b': 3, 'char_e': 4}

sentence18
因乌台诗案入狱，

sentence19
几死，因为写文章向朝廷诀别，
{'id': 'D0-S0-M0', 'string': '几', 'type': 'NUMBER', 'char_b': 0, 'char_e': 1}

sentence20
太皇太后曹氏、王安礼等人出面力挽，
{'id': 'D0-S0-M0', 'string': '王安礼', 'type': 'PER', 'char_b': 7, 'char_e': 10}

sentence21
神宗动心，苏轼终免一死，
{'id': 'D0-S0-M0', 'string': '苏轼终', 'type': 'PER', 'char_b': 5, 'char_e': 8}
{'id': 'D0-S0-M1', 'string': '一', 'type': 'NUMBER', 'char_b': 9, 'char_e': 10}



In [26]:
for d_i, d in enumerate(data):
    for q in d['QUESTIONS']:
        print(d_i)
        print(q['QTEXT_CN'])

0
上官均是否赞成王安石的新政改革?
1
苏东坡因为什么事件入狱几乎要死在狱中?
2
在「乌台诗案」中，有哪些人出面救了苏东坡？
3
苏东坡因「乌台诗案」被贬到什么地方？
4
宋神宗熙宁十月，黄河溃堤时，苏东坡人在哪里？
5
苏东坡被贬至黄州时，认识了哪些朋友？
6
西元1079年时徐州发生水灾的原因为何？
7
华尔街位于美国哪一个城市?
8
「占领华尔街」运动是由哪一个组织所发起?
9
「占领华尔街」运动的诉求为何?
10
「占领华尔街」运动延烧至美国哪些城市?
11
请列举出三项高雄的地标建物?
12
高雄市被列为国定古迹的建筑物是哪些?
13
高雄流行文化中心在哪里?
14
高雄有哪些大型购物中心?
15
高雄的大型购物中心有多少个?
16
高雄总共有多少座古迹?
17
「北美自由贸易协议2.0」的英文全名为何?
18
「北美自由贸易协议2.0」的英文简称为何?
19
「北美自由贸易协议2.0」的法文全名为何?
20
「北美自由贸易协议2.0」的法文简称为何?
21
「北美自由贸易协议2.0」的西班牙文全名为何?
22
「北美自由贸易协议2.0」的西班牙文简称为何?
23
「北美自由贸易协议2.0」的中文全名为何?
24
「北美自由贸易协议2.0」的中文简称为何?
25

9. 「北美自由贸易协议2.0」是哪三个国家之间的协议?
26
「北美自由贸易协议2.0」由哪三个国家的元首共同签署?
27
「北美自由贸易协议2.0」在何时签署?
28
「北美自由贸易协议2.0」在何地签署?
29
代表美国签署「北美自由贸易协议2.0」的人是谁?
30
代表墨西哥签署「北美自由贸易协议2.0」的人是谁?
31
代表加拿大签署「北美自由贸易协议2.0」的人是谁?
32
结婚男女对自己男性伴侣的称呼有哪些?
33
结婚男女对自己女性伴侣的称呼有哪些?
34
结婚后要如何称呼先生的父亲?
35
结婚后要如何称呼先生的母亲?
36
结婚后要如何称呼太太的父亲?
37
结婚后要如何称呼儿子的太太?(亲属关系称谓)
38
结婚后要如何称呼女儿的丈夫?(亲属关系称谓)
39
岳父指的是谁的父亲?
40
岳母指的是谁的母亲?
41
公公指的是谁的父亲?
42
婆婆指的是谁的母亲?
43
QQ平台是中国哪一家公司的？
44
QQ平台2017年推出那两款聊天机器人?
45
QQ小冰是哪一家公